In [1]:
import pandas as pd

In [2]:
inspections = pd.read_csv("../data/inspections.csv", index_col=0)

In [3]:
inspections.head(3)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,41158108,NICK'S GOURMET DELI,QUEENS,7415,DITMARS BOULEVARD,11370,7182788338,American,07/21/2015,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,Critical,11.0,NaN,NaN,09/27/2016,Cycle Inspection / Initial Inspection
1,41187577,HANSOL NUTRITION CENTER,QUEENS,16026,NORTHERN BOULEVARD,11358,7188880200,Korean,07/13/2016,Violations were cited in the following area(s).,06A,Personal cleanliness inadequate. Outer garment...,Critical,30.0,NaN,NaN,09/27/2016,Cycle Inspection / Initial Inspection
2,41705988,KURA,MANHATTAN,130,ST MARKS PLACE,10009,2122281010,Japanese,05/08/2013,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140Âº F.,Critical,27.0,NaN,NaN,09/27/2016,Pre-permit (Operational) / Initial Inspection


In [29]:
inspections['INSPECTION DATE'] = pd.to_datetime(inspections['INSPECTION DATE'])

In [47]:
# random_camis = inspections.sample().iloc[0]['CAMIS']
# inspections[inspections['CAMIS'] == random_camis].sort_values(by='INSPECTION DATE')['INSPECTION DATE']

Get initial inspection date.

In [4]:
inspections_f = inspections.groupby('CAMIS')\
                            .apply(lambda df: inspections[inspections['CAMIS'] == df['CAMIS'].iloc[0]]\
                                               .sort_values(by="INSPECTION DATE")\
                                               .iloc[0]\
                                               .drop('CAMIS')
                                  )\
                            .rename(columns={'INSPECTION DATE': 'INITIAL INSPECTION DATE'})

In [5]:
inspections_f.sample(1)

,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INITIAL INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
CAMIS,,,,,,,,,,,,,,,,,
50001937,TASTY CHICKEN,BROOKLYN,1687,86TH STREET,11214,7182591111,American,08/21/2013,Violations were cited in the following area(s).,15L,Smoke free workplace smoking policy inadequate...,Not Critical,NaN,NaN,NaN,09/27/2016,Smoke-Free Air Act / Initial Inspection


Not sure why, but this operation drops the `DBA` field. None others are affected. Maybe something with the indexing?

Let's take the most recent inspection date.

In [6]:
inspections_ff = inspections_f.copy()
inspections_ff['LATEST INSPECTION DATE'] = inspections_ff\
        .apply(lambda srs: inspections[inspections['CAMIS'] == srs.name]\
                                  .sort_values(by='INSPECTION DATE')\
                                  .iloc[-1]\
                                  ['INSPECTION DATE'],
              axis='columns')

Rename the `INSPECTION TYPE` to `INITIAL INSPECTION TYPE` (this flags new establishments from pre-existing ones, in the cotext of our dataset).

In [7]:
inspections_ff = inspections_ff.rename(columns={'INSPECTION TYPE': 'INITIAL INSPECTION TYPE'})

Checking the flags:

In [91]:
inspections_ff['INITIAL INSPECTION TYPE'].value_counts()

Cycle Inspection / Initial Inspection                          12537
Pre-permit (Operational) / Initial Inspection                   7630
Pre-permit (Non-operational) / Initial Inspection               2256
Cycle Inspection / Re-inspection                                 877
Administrative Miscellaneous / Initial Inspection                683
Smoke-Free Air Act / Initial Inspection                          281
Trans Fat / Initial Inspection                                   247
Pre-permit (Operational) / Re-inspection                          82
Trans Fat / Compliance Inspection                                 80
Trans Fat / Re-inspection                                         75
Inter-Agency Task Force / Initial Inspection                      53
Calorie Posting / Initial Inspection                              30
Trans Fat / Second Compliance Inspection                          27
Administrative Miscellaneous / Re-inspection                      27
Smoke-Free Air Act / Re-inspection

A lot of them are null.

In [93]:
inspections_ff['INITIAL INSPECTION TYPE'].isnull().astype(int).sum()

1094

Remember, this is new establishments that have not been inspected yet. That the numbers match up here is encouraging.

In [94]:
inspections['INSPECTION TYPE'].isnull().astype(int).sum()

1094

We reattach the lost DBA column.

In [95]:
inspections_ff.head(1)

,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INITIAL INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INITIAL INSPECTION TYPE,LATEST INSPECTION DATE
CAMIS,,,,,,,,,,,,,,,,,,
30075445,NaN,BRONX,1007.0,MORRIS PARK AVE,10462,7.188925e+09,Bakery,2013-06-01,Violations were cited in the following area(s).,16B,The original nutritional fact labels and/or in...,Not Critical,NaN,NaN,NaN,09/24/2016,Trans Fat / Compliance Inspection,2016-02-18


In [8]:
inspections_fff = inspections_ff.copy()
inspections_fff['DBA'] = inspections_fff.apply(lambda srs: 
                                                   inspections[inspections['CAMIS'] == srs.name]\
                                                   .iloc[0]['DBA'],
                                               axis='columns')

Prepend descriptors, to more easily distinguish things down the road.

In [9]:
inspections_fff.columns = ["DOHMH " + col for col in inspections_fff.columns]

In [105]:
inspections_fff.sample()

,DOHMH DBA,DOHMH BORO,DOHMH BUILDING,DOHMH STREET,DOHMH ZIPCODE,DOHMH PHONE,DOHMH CUISINE DESCRIPTION,DOHMH INITIAL INSPECTION DATE,DOHMH ACTION,DOHMH VIOLATION CODE,DOHMH VIOLATION DESCRIPTION,DOHMH CRITICAL FLAG,DOHMH SCORE,DOHMH GRADE,DOHMH GRADE DATE,DOHMH RECORD DATE,DOHMH INITIAL INSPECTION TYPE,DOHMH LATEST INSPECTION DATE
CAMIS,,,,,,,,,,,,,,,,,,
40374834,CASA BELLA,MANHATTAN,127.0,MULBERRY STREET,10013,2.124314e+09,Italian,2013-08-15,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,6.0,NaN,NaN,09/24/2016,Cycle Inspection / Initial Inspection,2016-04-26


Now we get information from Yelp!

In [22]:
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator
from yelp.errors import BusinessUnavailable
import os
import json

def import_credentials(filename='../data/yelp_credentials.json'):
    try:
        data = json.load(open(filename))
        return data
    except:
        raise IOError('This API requires Yelp credentials to work. Did you forget to define them?')

credentials = import_credentials()

auth = Oauth1Authenticator(
    consumer_key=credentials['consumer_key'],
    consumer_secret=credentials['consumer_secret'],
    token=credentials['token'],
    token_secret=credentials['token_secret']
)

client = Client(auth)

In [24]:
credentials

{'consumer_key': 'dkJPGu_jtTyHwsEgZIZN6g',
 'consumer_secret': 'lGsYFWNwi0QUNwN8XsNL4HmfvyE',
 'token': 'jMBCvOGvZ5IUDlsexgdRI_ELVzmo0q0i',
 'token_secret': '1g7Ui6r_iSED0qrX35cDzND3zCQ'}

In [ ]:
from tqdm import tqdm

In [41]:
def yelp_phone_fetch(num):
    """
    Performs the phone search described in notebook 02 to fetch information on the entity associated with a number.
    """
    if not num:
        return None
    else:
        try:
            business = client.phone_search(num).businesses[0]
            if business and business.location and business.location.coordinate:
                return {
                    'Yelp ID': business.id,
                    'Yelp Is Claimed': business.is_claimed,
                    'Yelp Is Closed': business.is_closed,
                    'Yelp Name': business.name,
                    'Yelp URL': business.url,
                    'Yelp Review Count': business.review_count,
                    'Yelp Categories': business.categories,
                    'Yelp Rating': business.rating,
                    'Yelp Address': business.location.address,
                    'Yelp Neighborhoods': business.location.neighborhoods,
                    'Yelp Latitude': business.location.coordinate.latitude,
                    'Yelp Longitude': business.location.coordinate.longitude,
                       }
            else:  # Partial information, skip.
                return None
        except IndexError:  # Phone search failed!
            return None

After some testing there appears to be a *very* significant miss rate. Remember, we're fuzzy matching phone numbers from one data set with phone numbers in another dataset. It's far from 100% that we'll get something back out. I'm interested in what percentage of the time we're successful.

In [17]:
def random_number_dba_tuple():
    random_number, random_dba = inspections_fff.sample().iloc[0][['DOHMH PHONE', 'DOHMH DBA']]
    random_number = str(int(random_number))
    return random_number, random_dba

In [15]:
random_number_dba_tuple()

(2125878880, 'MULTI TASTES DINER')

In [127]:
yelp_phone_fetch(random_number_dba_tuple()[0])

{'Yelp Address': ['41 E 11th St'],
 'Yelp Categories': [Category(name='Japanese', alias='japanese')],
 'Yelp ID': 'ootoya-greenwich-village-new-york',
 'Yelp Is Claimed': True,
 'Yelp Is Closed': False,
 'Yelp Latitude': 40.7333107,
 'Yelp Longitude': -73.9929962,
 'Yelp Name': 'Ootoya Greenwich Village',
 'Yelp Neighborhoods': ['Greenwich Village'],
 'Yelp Rating': 4.0,
 'Yelp Review Count': 135,
 'Yelp URL': 'https://www.yelp.com/biz/ootoya-greenwich-village-new-york?adjust_creative=dkJPGu_jtTyHwsEgZIZN6g&utm_campaign=yelp_api&utm_medium=api_v2_phone_search&utm_source=dkJPGu_jtTyHwsEgZIZN6g'}

Hey I've been there! Nice. More rigorously:

In [133]:
one_hundred_randoms = [random_number_dba_tuple() for i in range(100)]

In [138]:
testset = [yelp_phone_fetch(num) for num, placename in tqdm(one_hundred_randoms)]


100%|████████████████████████████████████████| 100/100 [00:27<00:00,  4.71it/s]


In [145]:
np.array([entity == None for entity in testset]).astype(int).sum()

19

19 misses in 100 randoms. So we're able to match 80% of the time. Not bad as far as fuzzy matches go.

If our data is missing at random (MAR), then we are happy, because this makes for a statistically valid sample of all restaurants in New York City&mdash;we can simply drop the other values.

But there's a high likelihood, in my professional opinion, that the data is missing not at a random (MNAR). I expect that it's more likely for a restaurant or eatery in a *poor* neighborhood to be missing the phone number information in Yelp! than one in a richer, whiter community. This is a severe under-reporting issue that will invalidate any conclusions we try to make using this data with regarding to "gentrification" and whatnot.

We'll need to validate the data geospatially. We'll do that next.

Run the full Yelp! API data through, one slice at a time.

In [150]:
len(inspections_fff)

26074

Oy vey. This will require two days of processing, since the API limit is 25000/day.

In [29]:
from tqdm import tqdm_notebook

In [28]:
del inspections
del inspections_f
del inspections_ff

In [38]:
inspections_fff['DOHMH PHONE'].iloc[2702:2705]

CAMIS
40788706    7187231080
40788884    __________
40788886    6462307208
Name: DOHMH PHONE, dtype: object

Uh, ok.

In [39]:
raw_yelp_5000 = [yelp_phone_fetch(int(num)) if pd.notnull(num) and str.isdigit(num) else None for num in tqdm_notebook(inspections_fff['DOHMH PHONE'][:5000])]

In [42]:
raw_yelp_5000_2 = [yelp_phone_fetch(int(num)) if pd.notnull(num) and str.isdigit(num) else None for num in tqdm_notebook(inspections_fff['DOHMH PHONE'][5000:10000])]

In [43]:
raw_yelp_5000_3 = [yelp_phone_fetch(int(num)) if pd.notnull(num) and str.isdigit(num) else None for num in tqdm_notebook(inspections_fff['DOHMH PHONE'][10000:15000])]

In [54]:
import pickle

with open("../data/raw_yelp_list.pkl", "wb") as f:
    pickle.dump(raw_yelp_5000+ raw_yelp_5000_2 + raw_yelp_5000_3, f)

In [48]:
raw_yelp_5000_4 = [yelp_phone_fetch(int(num)) if pd.notnull(num) and str.isdigit(num) else None for num in tqdm_notebook(inspections_fff['DOHMH PHONE'][15000:20000])]

KeyboardInterrupt: 

In [ ]:
raw_yelp_5000_5 = [yelp_phone_fetch(int(num)) if pd.notnull(num) and str.isdigit(num) else None for num in tqdm_notebook(inspections_fff['DOHMH PHONE'][20000:25000])]

In [ ]:
raw_yelp_5000_6 = [yelp_phone_fetch(int(num)) if pd.notnull(num) and str.isdigit(num) else None for num in tqdm_notebook(inspections_fff['DOHMH PHONE'][25000:])]

In [146]:
inspections_fff.to_csv("../data/inspections_flattened_initial.csv", encoding="utf-8")